In [15]:
import g_2
import anthropic
import random

with open('g_2/config.py') as config_file:
   key = config_file.read()
    
client = anthropic.Anthropic(api_key=key)

In [16]:
%run abDummy.ipynb

In [17]:
exp = Experiment(True)
exp.simQuadSpark()
log = Logbook(True)

A


In [18]:
tools = [{
            "name" : "getStatus",
            "description" : "Retrieves the status of the experiment.",
            "input_schema" : {
                "type" : "object",
                "properties" : {
                    "x" : {
                        "type" : "boolean",
                        "description" : "whether the status should be checked or not."
                    },
                }
            }
        }, 
        {
            "name" : "update",
            "description" : "Updates the internal status of the experiment.",
            "input_schema" : {
                "type" : "object",
                "properties" : {
                    "x" : {
                        "type" : "boolean",
                        "description" : "whether the status should be updated or not."
                    },
                }
            }
        }, 
         {
            "name" : "setKickers",
            "description" : "Controls the experiment's kickers.",
            "input_schema" : {
                "type" : "object",
                "properties" : {
                    "k1" : {
                        "type" : "number",
                        "description" : "voltage of kicker 1 in kV"
                    },
                     "k2" : {
                        "type" : "number",
                        "description" : "voltage of kicker 2 in kV"
                    }, 
                     "k3" : {
                        "type" : "number",
                        "description" : "voltage of kicker 3 in kV"
                    }, 
                     "status" : {
                        "type" : "boolean",
                        "description" : "if True the kickers pulse, if False kicker pulsing is inhibited. Setting status to true resets the kicker_spark flag"
                    }
                }
            }
        }, 
         {
            "name" : "setCollimator",
            "description" : "Insert or retract the collimators",
            "input_schema" : {
                "type" : "object",
                "properties" : {
                    "voltage" : {
                        "type" : "string",
                        "description" : "in means inserting collimeters, out means retracting"
                    }
                }
            }
        }, 
         {
            "name" : "setQuads",
            "description" : "Set the quad voltage and pulsing status.",
            "input_schema" : {
                "type" : "object",
                "properties" : {
                    "voltage" : {
                        "type" : "number",
                        "description" : "voltage in kV the quads are set to"
                    },
                    "status" : {
                        "type" : "string",
                        "description" : "pulsing sstatus, one of 'stopped', '1Hz', '5Hz', 'CCC'. Setting the status resets the quad spark flag. input 'CCC' for CCC trigger"
                    }
                }
            }
        }, 
         {
            "name" : "resetVacuum",
            "description" : "resets the vacuum to normal status",
            "input_schema" : {
             "type" : "object",
             "properties" : {
                 "x" : {
                     "type" : "boolean",
                     "description" : "Whether the vacuum should be reset or not."
                    },
                 }
            }
               
        }, 
        {
            "name" : "getInfo",
            "description" : "Returns the logbook information from a certain date",
            "input_schema" : {
             "type" : "object",
             "properties" : {
                 "date" : {
                     "type" : "string",
                     "description" : "The date of the entries that will be returned"
                    },
                 }
            }
        }, 
        {
            "name" : "getLatest",
            "description" : "Returns the latest logbook entry",
            "input_schema" : {
             "type" : "object",
             "properties" : {
                 "x" : {
                     "type" : "boolean",
                     "description" : "Whether the latest entry should be retrieved or not."
                    },
                 }
            }
        }, 
        {
            "name" : "getEntry",
            "description" : "Returns any logbook entries containing the keyword inputted",
            "input_schema" : {
             "type" : "object",
             "properties" : {
                 "keyword" : {
                     "type" : "string",
                     "description" : "the keyword for which you would like to get the entries containing"
                    },
                 }
            }
         
        }
         
        ]

In [19]:
def prompt_claude(prompt):
    messages= [{"role": "user", "content": prompt}]
    response = client.messages.create(
            model = "claude-3-5-sonnet-20240620",
            system = g_2.getSystemPrompt(prompt),
            max_tokens= 1000,
            messages = messages
        )
    #print(response.content[0].text)

    messages.append({"role": "assistant", "content": response.content})
    messages.append({"role": "user", "content": "You now have access to tools. If there is an action that can be completed using a tool, use it. If a tool is not required, respond as normal. Only call one tool at a time. Perform the instructions in order. If you encounter an issue, deal with it."})

    response2 = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            #system= "You now have access to tools. If there is an action that can be completed using a tool, use it. If a tool is not required, respond as normal.",
            max_tokens=1000,
            tools=tools,
            messages=messages
        )
    print(response2.content[0].text)
    i=response2
    c = 0

    while i.stop_reason == "tool_use":
        if c > 10:
            break
        tool_use = i.content[-1]
        tool_name = tool_use.name
        tool_input = tool_use.input
        #Add Claude's tool use call to messages:
        messages.append({"role": "assistant", "content": i.content})

        result = ""
        if tool_name == "getStatus":
            x = tool_input["x"]
            print("Claude wants to use get experiment status")
            exp.getStatus(True)
            result = exp.printStatus(True)
            print(result)

        elif tool_name == "update":
            x = tool_input["x"]
            print("Claude is updating experiment status")
            result = exp.update(x)

        elif tool_name == "setKickers":
            k1 = tool_input["k1"]
            k2 = tool_input["k2"]
            k3 = tool_input["k3"]
            status = tool_input["status"]
            print("Claude is setting kickers")
            result = exp.setKickers(k1, k2, k3, status)

        elif tool_name == "setCollimators":
            status = tool_input["status"]
            print("Claude is setting collimators")
            result = exp.setCollimators(status)

        elif tool_name == "setQuads":
            voltage = tool_input["voltage"]
            status = tool_input["status"]
            print("Claude is setting quads")
            result = exp.setQuads(voltage, status)

        elif tool_name == "resetVacuum":
            x = tool_input["x"]
            print("Claude is resetting vacuum")
            result = exp.resetVacuum(x)
            
        elif tool_name == "getInfo":
            date = tool_input["date"]
            print(f"Claude is getting log information for {date}")
            result = log.getInfo(date)

        elif tool_name == "getLatest":
            x = tool_input["x"]
            print("Claude is getting latest log entries")
            result = log.getLatest(x)
            print(result)

        elif tool_name == "getEntry":
            keyword = tool_input["keyword"]
            print(f"Claude is getting entries containing {keyword}")
            

        
            
        result = str(result)    
        #construct our tool_result message
        tool_response = {
            "role": "user",
            "content": [
                {
                "type": "tool_result",
                "tool_use_id": tool_use.id,
                "content": result
                }
            ]
            }
        messages.append(tool_response)
        #respond back to Claude
        i = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            messages=messages,
            max_tokens=1000,
            tools=tools
        )
        print(i.content[0].text)
        c = c+1


Exception in thread Thread-7:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1431, in run
    self.function(*self.args, **self.kwargs)
TypeError: Experiment.resetVacuum() missing 1 required positional argument: 'x'


In [20]:
prompt_claude("Perform normal shift operations based on experiment status")

In [ ]:
prompt_claude("get experiment status")

In [ ]:
prompt_claude("There was a quad spark in Q1")

In [ ]:
exp.simKickerSpark()
prompt_claude("Help! Something is wrong!!")

In [ ]:
exp.simTrolleyRun()
prompt_claude("Perform normal shift operations based on experiment status")

In [ ]:
exp.setQuads(10, "5Hz")
prompt_claude("Perform normal shift operations based on experiment status")

In [ ]:
exp.setQuads(10, "5Hz")
exp.getStatus(True)

In [ ]:
exp.getStatus(True)

In [ ]:
exp.__init__(True)

In [ ]:
exp.getStatus(True)

In [ ]:
exp.setQuads(10, "5Hz")
prompt_claude("Perform normal shift operations based on experiment status")

In [ ]:
%run abDummy.ipynb
log = Logbook(True)

In [ ]:
log.getInfo("07/15/2024")

In [ ]:
log.getLatest(True)

In [ ]:
log.getEntry("M31")

In [ ]:
print(log.getEntry("at"))

In [19]:
prompt_claude("get latest log entries")

I understand that I now have access to tools and should use them when appropriate. Thank you for the clarification. In this case, to get the latest log entries, we can use the `getLatest` function. Let me do that for you now.
Claude is getting latest log entries
The AP0 dump alarms became unsnoozed. Jim and ops snooze them for two weeks. They are snoozed until 08-05-24 at 0830.
Thank you for providing me with the tools. I've used the `getLatest` function to retrieve the most recent log entry. Here's a summary of the information we received:

The latest log entry reports:

1. The AP0 dump alarms became unsnoozed.
2. Jim and the operations team have snoozed these alarms.
3. The alarms are now snoozed for a period of two weeks.
4. Specifically, they will remain snoozed until August 5, 2024, at 08:30 AM.

This information is more current than what I initially provided, which was based on older data from January 2022. It's important to note that the experiment's log is being actively update

In [20]:
prompt_claude("Help! There was a quad spark. Is this a new problem?")

Certainly! To properly assess the current situation and provide you with accurate information, I'll need to check the status of the experiment. Let's start by using the getStatus tool to retrieve the current status.
Claude wants to use get experiment status

        vacuum: 1e-06
        quad_pulse: 5Hz
        quad_voltage: 13.1
        quad_spark: False
        quad_rf: True
        ctags: 0
        kicker_1: 7.39
        kicker_2: 7.78
        kicker_3: 7.77
        kicker_status: True
        kicker_spark: False
        ps_feedback: True
        self.ps_feedback_hold: False
        collimators: in
        pot: 1000000000000.0
        IBMS_signal_present: True
        inflector_current:2762.3
        magnet_current: 5173.5
        beam_permit: True
        beam_CDC: True
        m5_current: 789.3
        
Thank you for providing me with the tools to check the experiment status. Based on the current status, I can provide you with more accurate information about the quad spark situati

In [21]:
prompt_claude("Perform normal experiment operations. Monitor the logs for any abnormalities.")

Understood. I'll go through the instructions step by step, using the available tools when necessary. Let's start with checking the beam status.

First, let's get the current status of the experiment:
Claude wants to use get experiment status

        vacuum: 1e-06
        quad_pulse: 5Hz
        quad_voltage: 13.1
        quad_spark: False
        quad_rf: True
        ctags: 0
        kicker_1: 7.39
        kicker_2: 7.78
        kicker_3: 7.77
        kicker_status: True
        kicker_spark: False
        ps_feedback: True
        self.ps_feedback_hold: False
        collimators: in
        pot: 1000000000000.0
        IBMS_signal_present: True
        inflector_current:2762.3
        magnet_current: 5173.5
        beam_permit: True
        beam_CDC: True
        m5_current: 789.3
        
Based on this status, let's review the key points:

1. Beam status:
   - Beam permit is True, which is good.
   - CTAG (Coincidence Tag) is 0, which is abnormally low. It should be in the range of

In [24]:
prompt_claude("There was an issue with the experiment on july 19, 2024. What do we know about it?")

I apologize for the confusion in my previous response. You're right, I do have access to tools that can help us gather information about the experiment on July 19, 2024. Let's use these tools to find out what we know about any issues on that date.

First, let's retrieve the logbook information for July 19, 2024:
Claude is getting log information for 2024-07-19
Thank you for providing that information. It appears there were a couple of issues on July 19, 2024:

1. On July 18 at 13:57 (the day before), there was a trip of the MI-62 SB LCW Chipmunk (RD1216) due to a Missing Pulse.
2. On July 19 at 01:54 in the morning, the AP-30 D:H744 Chipmunk (RD2098) tripped on FSV (Full Scale Voltage).

Both of these are related to radiation detectors (Chipmunks). The log entry mentions that both radiation detectors were put into integrate mode and were successfully reset.

To get more context about these issues, let's check if there are any specific entries related to these problems:
Claude is gettin

In [31]:
prompt_claude("have there been any problems with the experiment lately?")

I understand. I'll use the available tools to check the current status of the experiment and provide you with up-to-date information. Let's start by checking the current status.
Claude wants to use get experiment status

        vacuum: 1e-06
        quad_pulse: stopped
        quad_voltage: 13.2
        quad_spark: True
        quad_rf: True
        ctags: 0
        kicker_1: 7.39
        kicker_2: 7.78
        kicker_3: 7.77
        kicker_status: True
        kicker_spark: False
        ps_feedback: True
        self.ps_feedback_hold: False
        collimators: in
        pot: 1000000000000.0
        IBMS_signal_present: True
        inflector_current:2762.3
        magnet_current: 5173.5
        beam_permit: True
        beam_CDC: True
        m5_current: 789.3
        
Thank you for providing me with the tools. I've used the getStatus function to check the current status of the experiment. Based on the results, I can provide you with an up-to-date summary of the experiment's condi

In [32]:
prompt_claude("Use log details to explain any recent problems?")

Certainly! I understand that I now have access to tools and should use them when appropriate. I'll proceed step by step, using tools when necessary and responding normally otherwise. I'll call only one tool at a time and follow the instructions in order.

To start, let's check the current status of the experiment to see if there are any ongoing issues or if the situation has changed since the log entries we discussed.
Claude wants to use get experiment status

        vacuum: 1e-06
        quad_pulse: stopped
        quad_voltage: 13.2
        quad_spark: True
        quad_rf: True
        ctags: 0
        kicker_1: 7.39
        kicker_2: 7.78
        kicker_3: 7.77
        kicker_status: True
        kicker_spark: False
        ps_feedback: True
        self.ps_feedback_hold: False
        collimators: in
        pot: 1000000000000.0
        IBMS_signal_present: True
        inflector_current:2762.3
        magnet_current: 5173.5
        beam_permit: True
        beam_CDC: True
      